A Router Chain permite realizar roteamento de prompts com base na entrada do usuário. Esse tipo de encadeamento é útil para lidar com diferentes tipos de perguntas ou resolver problemas distintos, enviando cada um para o processo correto. É uma técnica poderosa quando sua aplicação precisa lidar com fluxos mais dinâmicos e variados.


In [6]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


Fluxo 2

In [7]:
llm = ChatOpenAI(model="gpt-4o-mini")

Fluxo 3

In [8]:
prompt_acoes = ChatPromptTemplate.from_messages(
   [
       ("system", "Você é um especialista em investimentos em ações."),
       ("human", "{query}"),
   ]
)


prompt_renda_fixa = ChatPromptTemplate.from_messages(
   [
       ("system", "Você é um especialista em investimentos de renda fixa."),
       ("human", "{query}"),
   ]
)


Fluxo 4

In [9]:
chain_acoes = prompt_acoes | llm | StrOutputParser()
chain_renda_fixa = prompt_renda_fixa | llm | StrOutputParser()

In [10]:
route_system = "Roteie a pergunta do usuário para o especialista em ações ou renda fixa."
route_prompt = ChatPromptTemplate.from_messages(
   [
       ("system", route_system),
       ("human", "{query}"),
   ]
)

In [11]:
class RouteQuery(TypedDict):
   destination: Literal["acoes", "renda_fixa"]


route_chain = (
   route_prompt
   | llm.with_structured_output(RouteQuery)
   | itemgetter("destination")
)


chain = {
   "destination": route_chain,
   "query": lambda x: x["query"],
} | RunnableLambda(
   lambda x: chain_acoes if x["destination"] == "acoes" else chain_renda_fixa,
)


resultado = chain.invoke({"query": "Quais são os riscos de investir em ações de tecnologia?"})
print(resultado)

Investir em ações de tecnologia pode ser bastante atrativo, mas também envolve riscos significativos. Aqui estão alguns dos principais riscos associados a esse setor:

1. **Volatilidade do Mercado**: As ações de tecnologia tendem a ser mais voláteis do que ações de setores mais tradicionais. Mudanças rápidas nas condições de mercado podem levar a grandes oscilações nos preços.

2. **Inovação Rápida**: O setor de tecnologia evolui rapidamente. Empresas que hoje são líderes podem se tornar irrelevantes se não conseguirem inovar ou se adaptarem às novas tendências. A obsolescência de produtos e serviços é um risco constante.

3. **Concorrência Intensa**: O ambiente competitivo no setor de tecnologia é feroz. Novas startups podem surgir rapidamente e desafiar empresas estabelecidas, o que pode impactar suas participações de mercado e receitas.

4. **Mudanças Regulatórias**: O setor de tecnologia enfrenta uma crescente atenção regulatória, especialmente em questões como privacidade de dados